<a href="https://colab.research.google.com/github/Fabiorsi1979/DS-Comunidade/blob/main/SQL_revis%C3%A3o_geral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Treinamento SQL Revisão Geral

In [1]:
# conecção com o Banco de Dados que está no Google drive, na prática haverá outro caminho para coneção ou iremos retiorar informações direto do servidor.
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
#buscando e guardando em path o caminho de acesso ao bd.
path = '/content/drive/MyDrive/db_olist.sqlite'

In [3]:
# importando as bibliotecas que serão usadas
import sqlite3
from sqlite3 import Error
import pandas as pd

In [4]:
# criando a função que irá acessar o bd, fazer a consulta e guardar o retorno numa variável

def query_execute (*args):
   conecta = None
   try:
        # criando a conexão em conecta,
        # após criamos o cursor que irá rodar o bd linha a linha para fazer a consulta
        conecta = sqlite3.connect(path)
        cursor = conecta.cursor()

        # usandoa conexão e cursor efetuamos realmente a consulta
        # armazenamos o retorno da consulta numa variável
        cursor.execute(query)
        dados = cursor.fetchall()

        # fechamos a conexão e encerramos o cursor
        # assim evitamos erros para próxima consulta
        cursor.close()
        conecta.close()

        # agora transformamos os dados num modelo Dataframe,
        # de fácil manipulação em pandas
        dados = pd.DataFrame(data=dados)

   except Error as e:
          # apresenta o erro caso o mesmo ocorra, usando uma biblioteca do SQLite
          print (f'Erro {e} conexão')

          #retorna o resultado da função.
   return dados

#### 1. Qual a categoria que possui o produto com o maior número de dias entre a primeira compra da categoria e a sua data limite de entrega?("utilidades_domesticas","perfumaria","beleza_saude","telefonia")

In [5]:
query = """
    SELECT
         p.product_category_name AS categoria,
         MAX (CAST (JULIANDAY ( oi.shipping_limit_date ) - JULIANDAY (o.order_purchase_timestamp ) AS INTEGER)) AS maiorDia
    FROM products p LEFT JOIN order_items oi ON ( oi.product_id = p.product_id )
                    LEFT JOIN orders o ON ( oi.order_id = o.order_id )
    WHERE categoria IN ('perfumaria','telefonia','beleza_saude','utilidades_domesticas')
    GROUP BY categoria
    ORDER BY maiorDia DESC
"""
resultado = query_execute(query)
resultado.columns=["Categoria","Maior diferença"]
resultado.head()

,Categoria,Maior diferença
0,utilidades_domesticas,1056
1,perfumaria,46
2,telefonia,37
3,beleza_saude,36


#### 2.  Qual o código do pedido da categoria “utilidades domésticas” com 1283 dias entre a primeira compra dessa categoria e a data limite de envio do produto?

In [6]:
query = """
    SELECT
         o.order_id as id,
         CAST (JULIANDAY ( oi.shipping_limit_date ) - JULIANDAY (o.order_purchase_timestamp ) AS INTEGER) AS maiorDia
    FROM products p LEFT JOIN order_items oi ON ( oi.product_id = p.product_id )
                    LEFT JOIN orders o ON ( oi.order_id = o.order_id )
    WHERE p.product_category_name ='utilidades_domesticas' AND maiorDia > 1000
"""
resultado = query_execute(query)
print("Não encontrei esse número, portanto estou analisando os pedidos acima de 1000")
resultado.columns=["Código Pedido","Diferença > 1000"]
resultado.head(10)

Não encontrei esse número, portanto estou analisando os pedidos acima de 1000


,Código Pedido,Diferença > 1000
0,9c94a4ea2f7876660fa6f1b59b69c8e6,1056
1,13bdf405f961a6deec817d817f5c6624,1056
2,c2bb89b5c1dd978d507284be78a04cb2,1052
3,c2bb89b5c1dd978d507284be78a04cb2,1052


 #### 3. Qual a categoria com maior soma dos preços de produtos?

 #####    "automotivo","relogios_presentes","beleza_saude","cama_mesa_banho"


In [40]:
query = """
    SELECT
         p.product_category_name AS categoria,
         ROUND (SUM ( oi.price ),2)  AS maiorSomaprecos
    FROM products p LEFT JOIN order_items oi ON ( oi.product_id = p.product_id )
    WHERE categoria IN ("automotivo","relogios_presentes","beleza_saude","cama_mesa_banho")
    GROUP BY categoria ORDER BY maiorSomaprecos DESC
"""
resultado = query_execute(query)
resultado.columns=["Categoria","Maior Soma preços"]
resultado.head(4)

,Categoria,Maior Soma preços
0,beleza_saude,1258681.34
1,relogios_presentes,1205005.68
2,cama_mesa_banho,1036988.68
3,automotivo,592720.11


 #### 4. Qual o código do produto mais caro da categoria agro indústria & comercio?


In [8]:
query = """
SELECT
    tab1.id
FROM (SELECT
    p.product_id AS id,
    MAX ( oi.price ) AS maisCaro
FROM products p INNER JOIN order_items oi ON ( p.product_id = oi.product_id )
WHERE p.product_category_name LIKE 'agro%' AND p.product_category_name IS NOT NULL) AS tab1
"""
resultado = query_execute(query)
resultado.columns=["código do produto mais caro"]
resultado.head()

,código do produto mais caro
0,2b69866f22de8dad69c976771daba91c


#### 5. Qual a ordem correta das 3 categorias com os produtos mais caros?

In [38]:
query = """
SELECT
    p.product_category_name AS categoria,
    MAX ( oi.price ) AS maisCaro
FROM products p INNER JOIN order_items oi ON ( p.product_id = oi.product_id )
WHERE p.product_category_name IS NOT NULL
GROUP BY categoria ORDER BY maisCaro DESC LIMIT 5
"""
resultado = query_execute(query)
resultado.columns=["Categoria","Valores"]
resultado.head(3)

,Categoria,Valores
0,utilidades_domesticas,6735.0
1,pcs,6729.0
2,artes,6499.0


#### 6. Qual o valor dos produtos mais caros das categorias: bebes, flores e seguros e serviços, respectivamente:


In [11]:
query = """
SELECT
    p.product_category_name AS categoria,
    MAX ( oi.price ) AS maisCaro
FROM products p INNER JOIN order_items oi ON ( p.product_id = oi.product_id )
WHERE categoria IS NOT NULL AND categoria IN ("bebes","flores","seguros_e_servicos")
GROUP BY categoria ORDER BY maisCaro DESC
"""
resultado = query_execute(query)
resultado.columns = ["Categoria","Valor produto + caro"]
print()
print(f' US${resultado.iloc[0,1]}, US${resultado.iloc[2,1]} e US${resultado.iloc[1,1]}' )
print()
display(resultado)


 US$3899.0, US$65.9 e US$183.29



,Categoria,Valor produto + caro
0,bebes,3899.00
1,seguros_e_servicos,183.29
2,flores,65.90


#### 7. Quantos pedidos possuem um único comprador, 3 produtos e o pagamento foi dividido em 10 parcelas?


In [37]:
query = """
WITH contador AS (
        SELECT
              oi.order_id AS pedidos,
              o.customer_id AS clientesUnicos,
              op.payment_installments AS pagamentos,
              COUNT (oi.product_id ) AS Qtde_pedidos
        FROM order_items oi LEFT JOIN order_payments op ON ( op.order_id = oi.order_id )
                            LEFT JOIN orders o ON ( o.order_id = oi.order_id )
        GROUP BY pedidos
        HAVING pagamentos = 10 AND Qtde_pedidos = 3
)
SELECT
    COUNT ( contador.pedidos ) AS totalPedidos
FROM contador
"""
resultado = query_execute(query)
resultado.columns=["Total de Pedidos c/ Cliente=1, Produtos=3 e Parcelas 10"]
resultado.head()

,"Total de Pedidos c/ Cliente=1, Produtos=3 e Parcelas 10"
0,165


#### 8. Quantos pedidos foram parcelados em mais de 10 vezes ?

In [32]:
query = """
WITH pagMaior AS (
        SELECT
              op.order_id AS pedidos,
              op.payment_installments AS pagamentos
        FROM orders o LEFT JOIN order_payments op ON ( op.order_id = o.order_id )
        WHERE op.payment_installments > 10
)
SELECT
     COUNT ( p.pedidos ) as totalPedidos
FROM pagMaior p
"""
resultado = query_execute(query)
resultado.columns=["Pedidos parcelas > 10"]
resultado.head()

,Pedidos parcelas > 10
0,341


#### 9. Quantos clientes avaliaram o pedido com 5 estrelas?

In [21]:
query = """
SELECT
    COUNT ( o.customer_id ) AS numClientes
FROM orders o LEFT JOIN order_reviews orr ON ( orr.order_id = o.order_id )
WHERE orr.review_score = 5
"""
resultado = query_execute(query)
resultado.columns=["Qtde Clientes"]
resultado.head()

,Qtde Clientes
0,57420


#### 10. Quantos clientes avaliaram o pedido com 4 estrelas?

In [20]:
query = """
SELECT
    COUNT ( o.customer_id ) AS numClientes
FROM orders o LEFT JOIN order_reviews orr ON ( orr.order_id = o.order_id )
WHERE orr.review_score = 4
"""
resultado = query_execute(query)
resultado.columns=["Qtde Clientes"]
resultado.head()

,Qtde Clientes
0,19200


#### 11. Quantos clientes avaliaram o pedido com 3 estrelas?

In [24]:
query = """
SELECT
    COUNT ( o.customer_id ) AS numClientes
FROM orders o LEFT JOIN order_reviews orr ON ( orr.order_id = o.order_id )
WHERE orr.review_score = 3
"""
resultado = query_execute(query)
resultado.columns=["Qtde Clientes"]
resultado.head()

,Qtde Clientes
0,8287


#### 12.  Quantos clientes avaliaram o pedido com 2 estrelas?

In [23]:
query = """
SELECT
    COUNT ( o.customer_id ) AS numClientes
FROM orders o LEFT JOIN order_reviews orr ON ( orr.order_id = o.order_id )
WHERE orr.review_score = 2
"""
resultado = query_execute(query)
resultado.columns=["Qtde Clientes"]
resultado.head()

,Qtde Clientes
0,3235


#### 13. Quantos clientes avaliaram o pedido com 1 estrelas?

In [22]:
query = """
SELECT
    COUNT ( o.customer_id ) AS numClientes
FROM orders o LEFT JOIN order_reviews orr ON ( orr.order_id = o.order_id )
WHERE orr.review_score = 1
"""
resultado = query_execute(query)
resultado.columns=["Qtde Clientes"]
resultado.head()

,Qtde Clientes
0,11858


#### 14. No dia 2 de Outubro de 2016, qual era o valor da média móvel dos últimos 7 dias?


In [52]:
query = """
SELECT
    DATE ( o.order_purchase_timestamp )AS posicao,
    AVG ( oi.price ) OVER ( ORDER BY o.order_purchase_timestamp ASC
                                  ROWS BETWEEN 7 PRECEDING AND CURRENT ROW  ) AS mediaMovel7dias
FROM orders o LEFT JOIN order_items oi ON ( oi.order_id = o.order_id )
LIMIT 10
"""
resultado = query_execute(query)
resultado.columns=["Data","Média"]
resultado.loc[resultado['Data']=='2016-10-02']

,Data,Média
7,2016-10-02,52.48


#### 15. No dia 5 de Outubro de 2016 as 08:04:21, qual era o valor da média móvel dos últimos 14 dias?


In [51]:
query = """
SELECT
    o.order_purchase_timestamp,
    AVG ( oi.price ) OVER ( ORDER BY o.order_purchase_timestamp ASC
                                  ROWS BETWEEN 14 PRECEDING AND CURRENT ROW  ) AS mediaMovel14dias
FROM orders o LEFT JOIN order_items oi ON ( oi.order_id = o.order_id )
LIMIT 100
"""
resultado = query_execute(query)
resultado.columns=["Data","Média"]
resultado.loc[resultado['Data']=='2016-10-05 08:04:21']

,Data,Média
92,2016-10-05 08:04:21,79.835


#### 16.  Qual o código do produto da categoria agro indústria e comércio que está na 5 posição do ranking de produtos mais caros dessa categoria?

In [58]:
query = """
WITH exercicio16 AS (
    SELECT
    p.product_id AS codigoProduto,
    oi.price,
    DENSE_RANK () OVER ( ORDER BY oi.price DESC) AS produtosMaiscaros
  FROM order_items oi LEFT JOIN products p ON ( oi.product_id = p.product_id )
  WHERE p.product_category_name = 'agro_industria_e_comercio'
)
SELECT
  e16.codigoProduto,
  e16.produtosMaiscaros
FROM exercicio16 e16
"""
resultado = query_execute(query)
resultado.columns=["Codigo_Produto","Produtos_Caros"]
resultado.loc[resultado['Produtos_Caros']==5]['Codigo_Produto']

,Codigo_Produto
4,d5dbb4d9ecbbf2e312169e4c8f1b57f0


#### 17.  Qual o código do produto da categoria artes que está na posição 1 do ranking de produtos mais caros dessa categoria?

In [60]:
query = """
WITH exercicio17 AS (
    SELECT
    p.product_id AS codigoProduto,
    oi.price,
    DENSE_RANK () OVER ( ORDER BY oi.price DESC) AS produtosMaiscaros
  FROM order_items oi LEFT JOIN products p ON ( oi.product_id = p.product_id )
  WHERE p.product_category_name = 'artes'
  LIMIT 3
)
SELECT
  e17.codigoProduto,
  e17.produtosMaiscaros
FROM exercicio17 e17
"""
resultado = query_execute(query)
resultado.columns=["Codigo_Produto","Produtos_Caros"]
resultado.loc[resultado['Produtos_Caros']==1]['Codigo_Produto']

,Codigo_Produto
0,1bdf5e6731585cf01aa8169c7028d6ad


#### 18. Qual o valor da soma de todos que estão acima da posição 5 do ranking de produtos mais caros da categoria artes.


In [91]:
query = """
WITH exercicio18 AS (
    SELECT
    p.product_id AS codigoProduto,
    oi.price AS preco,
    DENSE_RANK () OVER ( ORDER BY oi.price DESC) AS rankMaiscaros
  FROM order_items oi LEFT JOIN products p ON ( oi.product_id = p.product_id )
  WHERE p.product_category_name = 'artes' AND p.product_category_name IS NOT NULL
  LIMIT 6
)
SELECT
  e18.codigoProduto,
  e18.preco,
  e18.rankMaiscaros
FROM exercicio18 e18
"""
resultado = query_execute(query)
resultado.columns=["Codigo_Produto","Valor Produtos","Rank"]
display(resultado)
print()
display(resultado.iloc[0:5,1])
print()
print(f"Valor da soma do 5º ao 1ª  = ", resultado.iloc[0:5,1].sum())

,Codigo_Produto,Valor Produtos,Rank
0,1bdf5e6731585cf01aa8169c7028d6ad,6499.00,1
1,9cf421049bcfaf6983bf0dfb40831a03,299.99,2
2,cbaff83e2c00a2b731d27276bbf52082,250.00,3
3,3aa071139cb16b67ca9e5dea641aaa2f,248.00,4
4,dc7e6c809b71f69934396f954df16806,239.90,5
5,b0694770dbbfebe2d48cd288cfce2f85,189.05,6


,Valor Produtos
0,6499.00
1,299.99
2,250.00
3,248.00
4,239.90



Valor da soma do 5º ao 1ª  =  7536.889999999999
